# <div align="center"><b> NLP2 - PRACTICO 1 </b></div>

<div align="right">📝 <em><small><font color='Gray'>Nota:</font></small></em></div>

<div align="right"> <em><small><font color='Gray'> La funcionalidad de visualización de jupyter notebooks en <a href="https://github.com/" target="_blank">github</a> es solamente un preview.</font></small></em> </div>

<div align="right"> <em><small><font color='Gray'> Para mejor visualización se sugiere utilizar el visualizador recomendado por la comunidad: <a href="https://nbviewer.org/" target="_blank">nbviewer</a></font></small></em> </div>

<div align="right"> <em><small><font color='Gray'> Puedes a acceder al siguiente enlace para ver este notebook en dicha página: <a href="https://nbviewer.org/github/brunomaso1/uba-mia/blob/mia-nlp2/mia-nlp2/Practico 1/research.ipynb">NLP2 - Practico 1</a></font></small></em> </div>

* * *

<style>
/* Limitar la altura de las celdas de salida en html */
.jp-OutputArea.jp-Cell-outputArea {
    max-height: 500px;
}
</style>

<!-- Colab -->
<!-- <div align="center"><img src="https://drive.google.com/uc?export=view&id=1QSNrTsz1hQbmZwpgwx0qpfpNtLW19Orm" width="600" alt="Figura 1: A data scientist is working on word generation using the Lord of the Rings lore. The image is dark and moody, with a focus on the scientist's computer screen. The screen displays a visualization the one ring, with a map of Middle Earth in the background. - Generada con DALL-E3"></div> -->

<div align="center"><img src="../resources/img1.jpeg" width="600" alt="Figura 1: A data scientist is sitting in front of a computer screen, intently focused on the task at hand. The room is dimly lit, with the only light coming from the computer screen. The neural network is displayed on the screen, with the data scientist working to. The nerual network is deepseek R1. - Generada con Microsoft Image Creator"></div>

<div align="center"><small><em>Figura 1: A data scientist is sitting in front of a computer screen, intently focused on the task at hand. The room is dimly lit, with the only light coming from the computer screen. The neural network is displayed on the screen, with the data scientist working to. The nerual network is deepseek R1. - Generada con Microsoft Image Creator</em></small></div>

<div align="center">✨Datos del proyecto:✨</div>

<p></p>

<div align="center">

| Subtitulo       | Practico 1 - NLP2                                                                                                                      |
| --------------- | -------------------------------------------------------------------------------------------------------------------------------------- |
| **Descrpción**  | CV Chatbot with Groq + Pinecone                                                                                                        |
| **Integrantes** | Bruno Masoller (brunomaso1@gmail.com)                                                                                                  |

</div>

🛻 <em><font color='MediumSeaGreen'>  Instalaciones: </font></em> 🛻

Asegurate de tener instalado poetry y estar en la carpeta donde se encuentra el archivo `pyproject.toml` para instalar las dependencias del proyecto.
Luego, instala las dependencias del proyecto ejecutando el siguiente comando en la terminal:

- bash:
```bash
poetry install
eval $(poetry env activate)
```

- powershell:
```powershell
poetry install
Invoke-Expression (poetry env activate)
```

✋ <em><font color='DodgerBlue'>Importaciones:</font></em> ✋

In [6]:
import os
import time
from dotenv import load_dotenv

# Pinecone
from pinecone import Pinecone, ServerlessSpec

# LangChain - Document Loaders
from langchain.document_loaders import PyPDFDirectoryLoader

# LangChain - Text Splitters
from langchain_text_splitters import RecursiveCharacterTextSplitter

# LangChain - Vector Stores
from langchain_pinecone import PineconeVectorStore

# LangChain - Embeddings
from langchain_community.embeddings import JinaEmbeddings

# LangChain - Chains
from langchain.chains import create_retrieval_chain, create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain

# LangChain - Prompts
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
)

# LangChain - Chat Models
from langchain_groq import ChatGroq

# LangChain - Messages
from langchain_core.messages import AIMessage, HumanMessage

🔧 <em><font color='tomato'>Configuraciones:</font></em> 🔧


In [7]:
DOT_ENV_PATH = "../.env"
load_dotenv(DOT_ENV_PATH)

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
JINA_AI_API_KEY = os.getenv("JINA_AI_API_KEY")
EMBEDDINGS_DIMENSION = 1024 
EMBEDDINGS_MODEL = "jina-embeddings-v3"
GROQ_MODEL = "llama-3.3-70b-versatile"
DIRECTORY_PATH = "../resources/cv/"

## Consinga

- Implementar un sistema de generación de texto (chatbot) que utilice la técnica de Retrieval-Augmented Generation (RAG). En este ejercicio, el chatbot será capaz de recuperar información de una base de datos (o un conjunto de documentos) y usarla para generar respuestas más completas, mejorando la calidad de las respuestas generadas.
- Entregables: Link a repo público y captura de video de chatbot consultando al CV del alumno. Utilizar librería como Streamlit (para capturar pantalla se puede utilizar OBS por ejemplo, es open source).

## Resolución

Para resolver este práctico, se utilizará la técnica de Retrieval-Augmented Generation (RAG) para crear un chatbot que pueda responder preguntas sobre el CV del alumno. El chatbot utilizará Pinecone como base de datos vectorial para almacenar y recuperar información relevante del CV. Además, se utilizará la API de Groq para generar respuestas a las preguntas del usuario. La gestión del flujo se maneja con LangChain (https://python.langchain.com/docs/how_to/), que permite integrar diferentes componentes de manera eficiente y sencilla. El flujo de trabajo se puede resumir en los siguientes pasos:

1. **Carga del CV**: Se cargará el CV del alumno en formato PDF y se extraerá el texto.
2. **División del texto**: Se dividirá el texto en fragmentos más pequeños para facilitar la recuperación de información.
3. **Indexación en Pinecone**: Se indexarán los fragmentos de texto en Pinecone para permitir la búsqueda eficiente.
4. **Probar la cadena de RAG**: Se probará la cadena de RAG para asegurarse de que funcione correctamente.
5. **Incorporar historial**: Se incorporará un historial de preguntas y respuestas para mejorar la experiencia del usuario.
6. **Limpieza**: Se limpiará el código y se preparará para la entrega final.
7. **Captura de video**: Se capturará un video del chatbot en funcionamiento, consultando el CV del alumno.

### Carga del CV

Inicialmente, se cargan todos los PDF de una carpeta. Para esto, se utiliza `langchain.document_loaders` + `PyPDFDirectoryLoader` que carga todos los documentos en el tipo de datos `Document`. 

In [ ]:
# Load the PDF files from the directory
loader = PyPDFDirectoryLoader(path=DIRECTORY_PATH, mode="single") # mode="single" -> cargar un solo documento por archivo PDF
                                                                  # mode="page" -> cargar una página por documento
documents = loader.load()

# Imprimir el número de documentos cargados. Se separa por páginas.
print(f"Number of documents loaded: {len(documents)}")

# Metadatos
print(f"Metadata of the first document: {documents[0].metadata}")

# Contenido
print(f"First document: {documents[0].page_content}")

Number of documents loaded: 1
Metadata of the first document: {'producer': 'Microsoft® Word para Microsoft 365', 'creator': 'Microsoft® Word para Microsoft 365', 'creationdate': '2025-04-08T18:48:43-03:00', 'author': 'Bruno Masoller', 'moddate': '2025-04-08T18:48:43-03:00', 'source': 'resources\\cv\\CV Bruno Masoller - Detallado.pdf', 'total_pages': 4}
First document: Página 1 de 4 
 
CURRICULUM VITAE 
DATOS PERSONALES: 
• NOMBRE: Bruno Martin Masoller Gancedo 
• DIRECCION: Montevideo, Uruguay 
• TELEFONO: +598 91 414 979 
• MAIL: brunomaso1@hotmail.com | brunomaso1@gmail.com 
• Linkedin: https://www.linkedin.com/in/brunomaso1/ 
FORMACION ACADEMICA: 
• Maestría en Inteligencia Artificial. Universidad de Buenos Aires. Facultad de Ingeniería. Fecha: EN 
CURSO. 
• Especialista en Inteligencia Artificial. Universidad de Buenos Aires. Facultad de Ingeniería. Fecha: 
EN CURSO. 
• Especialista en Ingeniería de Software. Universidad de la República. Facultad de Ingeniería. Fecha: 
2023. 
• Ing

### División del texto
<small><em>https://python.langchain.com/docs/concepts/text_splitters/#why-split-documents</em></small>

Una vez que tenemos los documentos, se dividen en fragmentos más pequeños con `langchain.text_splitter` + `RecursiveCharacterTextSplitter`. 

Más información sobre `RecursiveCharacterTextSplitter`:

> Text is naturally organized into hierarchical units such as paragraphs, sentences, and words. We can leverage this inherent structure to inform our splitting strategy, creating split that maintain natural language flow, maintain semantic coherence within split, and adapts to varying levels of text granularity. 

> 📝 <em><font color='Gray'>Nota:</font></em> También se puede utilizar un splitter semántico.
> Más información: https://python.langchain.com/docs/concepts/text_splitters/#semantic-meaning-based
> Tutorial: https://python.langchain.com/docs/how_to/semantic-chunker/

In [58]:
# Dividir el contenido en fragmentos
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=50,
    separators=["\n\n", "\n", "(?=\\.)", " ", ""],  # Separadores más robustos
    length_function=len,
)

# 'chunks' es una lista de objetos 'Document' de Langchain,
# donde cada 'Document' tiene el 'page_content' del fragmento
# y los metadatos del documento original
chunks = text_splitter.split_documents(documents)

# Textos unificados
text_chunks = [chunk.page_content for chunk in chunks]

print(f"Se generaron {len(chunks)} fragmentos.")

print(f"Tipo de chunk: {type(chunks[0])}") # Tiene la propiedad 'page_content' y 'metadata'.

print(f"Primer chunk: {chunks[0].page_content}")

Se generaron 62 fragmentos.
Tipo de chunk: <class 'langchain_core.documents.base.Document'>
Primer chunk: Página 1 de 4 
 
CURRICULUM VITAE 
DATOS PERSONALES: 
• NOMBRE: Bruno Martin Masoller Gancedo 
• DIRECCION: Montevideo, Uruguay 
• TELEFONO: +598 91 414 979


### Indexación en Pinecone
<small><em>https://python.langchain.com/api_reference/pinecone/vectorstores.html</em></small>

Para la indexación en Pinecone, con LangChain se puede utilizar los "VectorStore" que permiten almacenar y recuperar vectores de manera eficiente. En este caso se puede utilizar `PineconeVectorStore`.

In [59]:
# Creamos el indice en Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "nlp2-practico1"
namespace = "nlp2-practico1-namespace"

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        vector_type="dense",
        dimension=EMBEDDINGS_DIMENSION,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
        deletion_protection="disabled",
        tags={"environment": "development"},
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

print(f"Descripción del índice: {pc.describe_index(index_name)}")

Descripción del índice: {'deletion_protection': 'disabled',
 'dimension': 1024,
 'host': 'nlp2-practico1-22dypz1.svc.aped-4627-b74a.pinecone.io',
 'metric': 'cosine',
 'name': 'nlp2-practico1',
 'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
 'status': {'ready': True, 'state': 'Ready'},
 'tags': {'environment': 'development'},
 'vector_type': 'dense'}


In [60]:
# Obtenemos el indice
index = pc.Index(host=pc.describe_index(index_name).host)

Creamos el VectorStore con embeddings de Jina AI y lo indexamos en Pinecone. Para esto, se utiliza `JinaEmbeddings` para crear los embeddings de los fragmentos de texto y luego se indexan en Pinecone. Esto se hace utilizando la API de Jina AI, pero se puede lograr lo mismo utilizando SentenceTransformers.

Links: 
- [Embbedings de Jina AI - LangChain](https://python.langchain.com/docs/integrations/text_embedding/jina/)
- [Sentence Transformers Embbedings - LangChain](https://python.langchain.com/docs/integrations/text_embedding/sentence_transformers/)
- [Custom Embbedings - LangChain](https://python.langchain.com/docs/how_to/custom_embeddings/)

<details>
  <summary>Ejemplo utilizando HuggingFaceEmbbedings</summary>
  
```python
    from langchain_huggingface.embeddings import HuggingFaceEmbeddings

    text_embbedings = HuggingFaceEmbeddings(model_name="jinaai/jina-embeddings-v3", model_kwargs={"trust_remote_code": True})

    text = "This is a test document."
    query_result = text_embbedings.embed_query(text)

    # show only the first 100 characters of the stringified vector
    print(str(query_result)[:100] + "...")

    doc_result = text_embbedings.embed_documents([text, "This is not a test document."])
    print(str(doc_result)[:100] + "...")

    # Inicializa el vector store igual que antes
    vector_store = PineconeVectorStore(
        index=index,
        embedding=text_embbedings,
        namespace=namespace
    )
```

</details>

<details>
  <summary>Ejemplo utilizando Custom Embedding</summary>
  
```python
    from transformers import AutoModel
    from torch.nn.functional import normalize
    from langchain_core.embeddings import Embeddings

    class JinaLocalEmbeddings(Embeddings):
        def __init__(self, model_name):
            self.model = AutoModel.from_pretrained(model_name)
            self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        
        def embed_documents(self, texts):
            inputs = self.tokenizer(
                texts, 
                padding=True, 
                truncation=True, 
                return_tensors="pt",
                max_length=8192  # Máximo contexto de Jina v3
            )
            outputs = self.model(**inputs)
            embeddings = normalize(outputs.last_hidden_state.mean(dim=1), p=2, dim=1)
            return embeddings.detach().numpy().tolist()
        
        def embed_query(self, text):
            return self.embed_documents([text])[0]

    # Uso
    text_embeddings = JinaLocalEmbeddings("jinaai/jina-embeddings-v3")
```

</details>

Utilizamos la API de Jina AI:

In [61]:
# Creamos el modelo de embeddings de Jina
text_embeddings = JinaEmbeddings(
    jina_api_key=JINA_AI_API_KEY, model_name=EMBEDDINGS_MODEL
)

In [62]:
# Creamos el vector store de Pinecone
vector_store = PineconeVectorStore.from_documents(
    documents=chunks,
    embedding=text_embeddings,
    index_name=index_name,
    namespace=namespace
) 

In [63]:
vector_store2 = PineconeVectorStore.from_existing_index(
    embedding=text_embeddings,
    index_name=index_name,
    namespace=namespace
)

Realizamos un test de la búsqueda:

In [64]:
results = vector_store.similarity_search(
    query="¿Cuáles son los lenguajes de programación más utilizados en el CV?",
    k=3,
    namespace=namespace,
)

print(f"Resultados de la búsqueda: {results}")
print(f"Contenido del primer resultado: {results[0].page_content}")

Resultados de la búsqueda: [Document(id='4baec4c6-d1fd-4e4d-ba08-75acf019f7c1', metadata={'author': 'Bruno Masoller', 'creationdate': '2025-04-08T18:48:43-03:00', 'creator': 'Microsoft® Word para Microsoft 365', 'moddate': '2025-04-08T18:48:43-03:00', 'producer': 'Microsoft® Word para Microsoft 365', 'source': '..\\resources\\cvs_pdf\\CV Bruno Masoller - Detallado.pdf', 'total_pages': 4.0}, page_content='• Ingeniería de software \n• MLOps \n• SQL \n• Python \n• Java \n• Quarkus \n• Scikit-learn \n• SAP Process \nOrchestration \n• Angular \n• Typescript \n• Ionic \n \nINTERESES Y ÁREAS DE INVESTIGACIÓN:'), Document(id='3c6561cc-8474-4c8a-b14c-5a5192ee2d7a', metadata={'author': 'Bruno Masoller', 'creationdate': '2025-04-08T18:48:43-03:00', 'creator': 'Microsoft® Word para Microsoft 365', 'moddate': '2025-04-08T18:48:43-03:00', 'producer': 'Microsoft® Word para Microsoft 365', 'source': '..\\resources\\cvs_pdf\\CV Bruno Masoller - Detallado.pdf', 'total_pages': 4.0}, page_content='• Ingen

Obtenemos el `retriever` de LangChain para poder realizar la búsqueda de los fragmentos de texto más relevantes en función de la pregunta del usuario. Más información: https://python.langchain.com/docs/concepts/retrievers/

In [65]:
vector_store_retriever = vector_store.as_retriever(search_kwargs={"k": 5})

In [66]:
# Realizamos una prueba de búsqueda
query = "¿Cuáles son los lenguajes de programación más utilizados en el CV?"
results = vector_store_retriever.invoke(query)
print(f"Resultados de la búsqueda: {results}")

Resultados de la búsqueda: [Document(id='4baec4c6-d1fd-4e4d-ba08-75acf019f7c1', metadata={'author': 'Bruno Masoller', 'creationdate': '2025-04-08T18:48:43-03:00', 'creator': 'Microsoft® Word para Microsoft 365', 'moddate': '2025-04-08T18:48:43-03:00', 'producer': 'Microsoft® Word para Microsoft 365', 'source': '..\\resources\\cvs_pdf\\CV Bruno Masoller - Detallado.pdf', 'total_pages': 4.0}, page_content='• Ingeniería de software \n• MLOps \n• SQL \n• Python \n• Java \n• Quarkus \n• Scikit-learn \n• SAP Process \nOrchestration \n• Angular \n• Typescript \n• Ionic \n \nINTERESES Y ÁREAS DE INVESTIGACIÓN:'), Document(id='038564ff-5b42-4488-9bfe-67c7b687f9fc', metadata={'author': 'Bruno Masoller', 'creationdate': '2025-04-08T18:48:43-03:00', 'creator': 'Microsoft® Word para Microsoft 365', 'moddate': '2025-04-08T18:48:43-03:00', 'producer': 'Microsoft® Word para Microsoft 365', 'source': '..\\resources\\cvs_pdf\\CV Bruno Masoller - Detallado.pdf', 'total_pages': 4.0}, page_content='• Ingen

### Probar la cadena de RAG
<small><em>https://python.langchain.com/docs/how_to/#langchain-expression-language-lcel</em></small>

In [67]:
# Instanciamos el modelo de Groq
chat = ChatGroq(model="llama-3.1-8b-instant")

> 📝 <em><font color='Gray'>Nota:</font></em> combine_docks_chain:
> Es el componente encargado de combinar los documentos recuperados (por el retriever) con el prompt original del usuario para generar una respuesta coherente. Básicamente:
> 1. Recibe la pregunta del usuario (ej: "¿Cuáles son los lenguajes de programación más utilizados en el CV?")
> 2. Recibe los documentos/chunks relevantes del vector store.
> 3. Combina ambos elementos en un formato entendible para el LLM.
> 4. Genera una respuesta final basada en el contexto.

Primeramente, debemos definir un prompt template:

In [68]:
from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate
system_prompt = """
Responde la pregunta del usuario basándote en el siguiente contexto:

<contexto>
{context}
</contexto>

Si no hay información relevante en el contexto, di "No sé la respuesta".
"""

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(system_prompt),
        HumanMessagePromptTemplate.from_template("{input}"),
    ]
)

Donde:
- Se incluye un placeholder `context` para los documentos recuperados por el retriever (esto está en la documentación de combine_doc_chain).

Creamos el `combine_docs_chain` con el prompt template y el modelo de Groq:

In [69]:
combine_docs_chain = create_stuff_documents_chain(
    llm=chat, prompt=prompt
)

Creamos `create_retrieval_chain` para crear la cadena de RAG:

In [70]:
retrieval_chain = create_retrieval_chain(
    retriever=vector_store_retriever,
    combine_docs_chain=combine_docs_chain
)

Probamos la cadena:

In [71]:
response = retrieval_chain.invoke(
    {"input": "¿Cuáles son los lenguajes de programación más utilizados en el CV?"}
)

print(f"Respuesta: {response['answer']}")

Respuesta: Basándome en el contexto proporcionado, los lenguajes de programación más utilizados mencionados en el CV son:

1. Python
2. Java
3. Typescript

Estos lenguajes de programación se mencionan varias veces en el contexto como intereses y áreas de investigación, lo que sugiere que el titular del CV tiene una buena conocimiento y experiencia en ellos.


In [72]:
response

{'input': '¿Cuáles son los lenguajes de programación más utilizados en el CV?',
 'context': [Document(id='4baec4c6-d1fd-4e4d-ba08-75acf019f7c1', metadata={'author': 'Bruno Masoller', 'creationdate': '2025-04-08T18:48:43-03:00', 'creator': 'Microsoft® Word para Microsoft 365', 'moddate': '2025-04-08T18:48:43-03:00', 'producer': 'Microsoft® Word para Microsoft 365', 'source': '..\\resources\\cvs_pdf\\CV Bruno Masoller - Detallado.pdf', 'total_pages': 4.0}, page_content='• Ingeniería de software \n• MLOps \n• SQL \n• Python \n• Java \n• Quarkus \n• Scikit-learn \n• SAP Process \nOrchestration \n• Angular \n• Typescript \n• Ionic \n \nINTERESES Y ÁREAS DE INVESTIGACIÓN:'),
  Document(id='038564ff-5b42-4488-9bfe-67c7b687f9fc', metadata={'author': 'Bruno Masoller', 'creationdate': '2025-04-08T18:48:43-03:00', 'creator': 'Microsoft® Word para Microsoft 365', 'moddate': '2025-04-08T18:48:43-03:00', 'producer': 'Microsoft® Word para Microsoft 365', 'source': '..\\resources\\cvs_pdf\\CV Bruno Ma

### Incorporar historial
<small><em>https://python.langchain.com/v0.1/docs/use_cases/question_answering/chat_history/</em></small>

> ⚠️ <em><font color='gold'>Problemas detectados:</font></em> Actualmente la versión oficial de para incorporar historial es mediante LangGraph:
> - [Página LangGrahp](https://langchain-ai.github.io/langgraph/)
> - [Tutorial](https://python.langchain.com/docs/how_to/qa_chat_history_how_to/)

Creamos un retriever (que es un LLM) que utiliza el historial para reformular la pregunta y generar una respuesta más completa, con el historial:

In [74]:
reformulate_history_prompt = """
Dado un historial de un chat y la última pregunta del usuario, que podría referenciar al contexto del historial, crea una pregunta independiente que pueda entenderse sin el historial. NO respondas a la pregunta; simplemente reformula si es necesario; de lo contrario, devuélvala tal como está.
"""

# Prompt para reformular la pregunta usando el historial
retriever_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(reformulate_history_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{input}"),
    ]
)

# Retriever que usa el historial
history_aware_retriever = create_history_aware_retriever(
    llm=chat, retriever=vector_store_retriever, prompt=retriever_prompt
)

Modificamos el `combine_docs_chain` para que utilice el nuevo prompt (que utiliza el historial):

In [75]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(system_prompt),
        MessagesPlaceholder("chat_history"),
        HumanMessagePromptTemplate.from_template("{input}"),
    ]
)

combine_docs_chain = create_stuff_documents_chain(llm=chat, prompt=qa_prompt)

Creamos la cadena final utilizando `create_retrieval_chain` (esta cadena utiliza el prompt que reformula la pregunta y luego va a buscar los documentos relevantes en el vector store):

In [ ]:
# Cadena principal
retrieval_chain = create_retrieval_chain(
    retriever=history_aware_retriever, combine_docs_chain=combine_docs_chain
)

Finalmente, probamos la cadena:

In [85]:
# Primera pregunta
response1 = retrieval_chain.invoke(
    {"input": "¿Cuáles son los lenguajes de programación más utilizados en el CV?", "chat_history": []}
)
print("Respuesta 1:", response1['answer'])

# Segunda pregunta (usa el historial)
response2 = retrieval_chain.invoke(
    {
        "input": "Del primer lenguaje de programación, ¿existe algún proyecto destacado?",
        "chat_history": [
            HumanMessage(content=response1['input']),
            AIMessage(content=response1['answer']),
        ],
    }
)
print("Respuesta 2:", response2['answer'])

print("Historial de la conversación:")
print("-" * 20)
print(response1)
print("-" * 20)
print(response2)

Respuesta 1: Basándome en el contexto proporcionado, los lenguajes de programación más utilizados en el campo de la ingeniería de software y MLOps mencionados son:

1. Python
2. Java
3. Typescript
Respuesta 2: Sí, en el contexto proporcionado, hay varios proyectos de trabajos realizados durante la carrera de Especialización en Inteligencia Artificial (UBA) que utilizan Python. Sin embargo, no hay información específica sobre proyectos destacados en el contexto proporcionado.

Pero puedo mencionar algunos ejemplos de proyectos destacados en la comunidad de IA que utilizan Python, como:

1. TensorFlow (librería de aprendizaje automático)
2. Keras (librería de aprendizaje automático)
3. Scikit-learn (biblioteca de aprendizaje automático)
4. OpenCV (biblioteca de visión computacional)

Estos proyectos son ampliamente utilizados en la comunidad de IA y Machine Learning, y son una buena oportunidad para aplicar conocimientos en Python.
Historial de la conversación:
--------------------
{'inp

### Limpieza

In [89]:
# Eliminamos el índice
try:
    pc.delete_index(index_name)
    print(f"Índice {index_name} eliminado correctamente.")
except Exception as e:
    print(f"Error al eliminar el índice {index_name}: {e}")

# Listamos los índices restantes
pc.list_indexes()

Error al eliminar el índice nlp2-practico1: (404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-01', 'x-cloud-trace-context': '89abf986a5d173aba1ae403b1e313f69', 'date': 'Wed, 16 Apr 2025 19:12:55 GMT', 'server': 'Google Frontend', 'Content-Length': '89', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"NOT_FOUND","message":"Resource nlp2-practico1 not found"},"status":404}



[]

### Ejecución de aplicación

In [ ]:
!streamlit run app.py

### Video de resolución

A continuación se muestra un video de la resolución del práctico, donde se puede ver el chatbot funcionando y respondiendo preguntas sobre el CV del alumno. El código de la aplicación se encuentra en: [codigo fuente](./app.py)

In [ ]:
%%HTML
<video width="800"  controls>
  <source src="../resources/practico1.mp4" type="video/mp4">
</video>